In [ ]:
# !pip install psycopg2

## Import Required Packages

In [ ]:
import pandas as pd
import numpy as np
import psycopg2
from sqlalchemy import create_engine
import time

## Conenct to Database

In [ ]:
# Connect to the PostgreSQL database
host = '173.249.1.23'
port = '5432'
db = 'ph_api'
user = 'postgres'
password = 'Password'
conn_str = f'postgresql://{user}:{password}@{host}:{port}/{db}'
engine = create_engine(conn_str)



## Evaluate metrics - Ratings and Feedback

In [ ]:
# Specify the table name
table_name = 'feedback_feedback'

# Establish a connection and execute the query
conn = psycopg2.connect(database=db, user=user, password=password, host=host, port=port)
cursor = conn.cursor()
cursor.execute(f"SELECT * FROM {table_name}")

# Fetch all rows from the cursor
rows = cursor.fetchall()

# Create a pandas DataFrame from the fetched rows
df = pd.DataFrame(rows, columns=[desc[0] for desc in cursor.description])

# Close the cursor and the connection
cursor.close()
conn.close()

# # Display the DataFrame
# df.head()

In [ ]:
len(df)

52

In [ ]:
from textblob import TextBlob

df['sentiment_polarity'] = df['feedback'].apply(lambda x: TextBlob(x).sentiment.polarity)


In [ ]:
df.head()

,id,test_id,rating,feedback,status,created_date,user_id,sentiment_polarity
0,1,f759c6f0-e47d-4971-8d29-81e561c9e077,4,best,Active,2023-06-14 17:01:55.938720+02:00,4,1.00
1,2,f759c6f0-e47d-4971-8d29-81e561c9e077,5,Great experience!,Active,2023-06-14 17:06:34.360191+02:00,3,1.00
2,3,f759c6f0-e47d-4971-8d29-81e561c9e077,4,very useful,Active,2023-06-14 17:07:26.164747+02:00,4,0.39
3,4,cb2b8364-aba6-493a-9955-e4a3b2bfe903,5,best,Active,2023-06-14 21:16:23.916749+02:00,3,1.00
4,5,17269abb-a8d6-411d-a832-a1269f87fb92,5,very helpful,Active,2023-06-14 22:49:15.385489+02:00,4,0.20


In [ ]:
# Categorize feedback based on sentiment
df['sentiment_category'] = df['sentiment_polarity'].apply(lambda x: 'Positive' if x > 0 else 'Negative' if x < 0 else 'Neutral')

# Calculate the count of feedback entries by sentiment category
sentiment_counts = df['sentiment_category'].value_counts()

# Calculate the percentage of feedback entries by sentiment category
sentiment_percentages = df['sentiment_category'].value_counts(normalize=True) * 100

# Display the sentiment distribution
print(sentiment_counts)
print(sentiment_percentages)

Positive    37
Neutral     12
Negative     3
Name: sentiment_category, dtype: int64
Positive    71.153846
Neutral     23.076923
Negative     5.769231
Name: sentiment_category, dtype: float64


In [ ]:
# Calculate the count of each rating
rating_counts = df['rating'].value_counts().sort_index()

# Display the count of each rating
for rating in range(1, 6):
    count = rating_counts.get(rating, 0)
    print(f"Rating {rating}: {count}")

Rating 1: 3
Rating 2: 4
Rating 3: 6
Rating 4: 16
Rating 5: 23


Positive Recommendation: A rating of 4 or 5.</br>
True Positive (TP): A positive recommendation that matches an actual positive video. </br>
False Positive (FP): A positive recommendation that does not match an actual positive video.</br>
False Negative (FN): An actual positive video that was not recommended.</br>
Precision: The ratio of true positives to the sum of true positives and false positives.</br>
Recall: The ratio of true positives to the sum of true positives and false negatives.

In [ ]:
# Let's convert the rating column to numeric type for easier calculations
df['rating'] = pd.to_numeric(df['rating'])

# Define positive recommendations as ratings of 4 or 5
df['positive_recommendation'] = df['rating'].apply(lambda x: x >= 4)

# Calculate True Positives (TP), False Positives (FP), and False Negatives (FN)
TP = df[df['positive_recommendation'] == True].shape[0]
FP = df[df['positive_recommendation'] == False].shape[0]
FN = df[(df['rating'] >= 4) & (df['positive_recommendation'] == False)].shape[0]

# Calculate Precision and Recall
precision = TP / (TP + FP)
recall = TP / (TP + FN)

print(f"Precision: {precision:.2f}")
print(f"Recall: {recall:.2f}")

Precision: 0.75
Recall: 1.00


Precision: 0.76
Precision is a measure of the accuracy of positive recommendations made by the model. In this case, a precision of 0.76 means that out of all the recommendations classified as positive by the model, 76% of them were actually correct and matched with positive ratings given by users. In other words, the model has a relatively high accuracy in correctly identifying positive recommendations.

Recall: 1.00
Recall is a measure of the completeness of positive recommendations made by the model. A recall of 1.00 means that the model was able to capture all the positive recommendations present in the evaluation data. It indicates that the model did not miss any positive recommendations.

Considering both precision and recall together, a high precision score suggests that the model is accurate in identifying positive recommendations, while a high recall score indicates that the model is comprehensive in capturing positive recommendations. In this case, the model has high recall (capturing all positive recommendations) but slightly lower precision (some recommendations classified as positive were incorrect).

Overall, these scores suggest that the model is able to identify a large proportion of positive recommendations but has room for improvement in reducing false positive recommendations. It would be beneficial to further analyze the false positive cases to understand the reasons behind them and refine the model accordingly.






# NDCG

## Calculate NDCG without considering user-level rankings

In [ ]:
# Define relevance scores based on ratings (assuming higher rating indicates higher relevance)
df['relevance'] = df['rating']

# Sort the DataFrame by rating in descending order
df_sorted = df.sort_values('rating', ascending=False)

# Calculate the Discounted Cumulative Gain (DCG)
df_sorted['rank'] = df_sorted.index + 1
df_sorted['dcg'] = (2 ** df_sorted['relevance'] - 1) / np.log2(df_sorted['rank'] + 1)

# Calculate the Ideal Discounted Cumulative Gain (IDCG)
df_sorted['idcg'] = (2 ** df_sorted['relevance'] - 1) / np.log2(df_sorted['rank'].max() + 1)

# Calculate the Normalized Discounted Cumulative Gain (NDCG)
df_sorted['ndcg'] = df_sorted['dcg'] / df_sorted['idcg']

# Calculate the average NDCG
ndcg_avg = df_sorted['ndcg'].mean()

print(f"Average NDCG: {ndcg_avg:.4f}")

Average NDCG: 1.4593


## Calculate NDCG with considering user-level rankings

In [ ]:
# Define relevance scores based on ratings (assuming higher rating indicates higher relevance)
df['relevance'] = df['rating']

# Sort the DataFrame by user_id and rating in descending order
df_sorted = df.sort_values(['user_id', 'rating'], ascending=[True, False])

# Group the DataFrame by user_id and calculate the Discounted Cumulative Gain (DCG)
df_sorted['rank'] = df_sorted.groupby('user_id').cumcount() + 1
df_sorted['dcg'] = (2 ** df_sorted['relevance'] - 1) / np.log2(df_sorted['rank'] + 1)

# Calculate the Ideal Discounted Cumulative Gain (IDCG)
df_sorted['idcg'] = (2 ** df_sorted['relevance'] - 1) / np.log2(df_sorted['rank'].max() + 1)

# Group the DataFrame by user_id and calculate the Normalized Discounted Cumulative Gain (NDCG)
df_sorted['ndcg'] = df_sorted['dcg'] / df_sorted['idcg']

# Calculate the average NDCG across all users
ndcg_avg = df_sorted.groupby('user_id')['ndcg'].mean().mean()

print(f"Average NDCG: {ndcg_avg:.4f}")

Average NDCG: 4.0349


we assume that higher ratings indicate higher relevance. We calculate the Discounted Cumulative Gain (DCG) by assigning ranks to the recommendations within each user and applying the formula (2 ** relevance - 1) / log2(rank + 1) to calculate the DCG value for each recommendation.

In [ ]:
# Define relevant items based on ratings (assuming higher rating indicates relevance)
df['relevant'] = df['rating'].apply(lambda x: 1 if x >= 4 else 0)

# Sort the DataFrame by rating in descending order
df_sorted = df.sort_values('rating', ascending=False)

# Calculate the cumulative sum of relevant items and rank
df_sorted['cumulative_relevant'] = df_sorted['relevant'].cumsum()
df_sorted['rank'] = df_sorted.index + 1

# Calculate precision and average precision
df_sorted['precision'] = df_sorted['cumulative_relevant'] / df_sorted['rank']
df_sorted['average_precision'] = df_sorted['precision'] * df_sorted['relevant']

# Calculate Mean Average Precision (MAP)
map_avg = df_sorted['average_precision'].sum() / df_sorted['relevant'].sum()

print(f"Mean Average Precision (MAP): {map_avg:.4f}")

Mean Average Precision (MAP): 2.3845


The Mean Average Precision (MAP) value of 2.5866 indicates the overall quality of your recommendation engine based on the provided feedback data.</br>
In this case, a MAP value of 2.5866 suggests that, on average, your recommendation engine provides recommendations that are somewhat relevant to the users. However, since the MAP value is below 5, it indicates that there is room for improvement in terms of precision and the ability to recommend highly relevant items

In [ ]:
# Group the data by rating and concatenate the feedback text
feedback_by_rating = df.groupby('rating')['feedback'].apply(lambda x: ' '.join(x))

# Analyze the feedback for each rating
for rating, feedback in feedback_by_rating.items():
    print(f"Rating {rating}:")
    print(feedback, ',')
    print( )


Rating 1:
. . did not understand CBT video ,

Rating 2:
no time want more stress related videos ok type like 3 videos ,

Rating 3:
useful insightful and not highly relevant to my field of interest Not happy with recommendations Daniel Seigal video is awesome tedx videos so nice could have more relevant to my problem ,

Rating 4:
best very useful very useful helpful useful useful great recommendations great recommendations useful This is working well good content. can add few more could have been more relevant based on my profession long videos The recommendations were diverse and introduced me to new and interesting options. Little long videos, I would have preferred slightly short videos. But yes, interesting videos will complete later ,

Rating 5:
Great experience! best very helpful very useful good recommendations very helpful thanks useful very useful useful Recommendations made sense to me. The summary format is not readable. Please bring human connect in your summary. thanks for 

# Video Likes and Dislikes

In [ ]:
# Specify the table name
table_name = 'assesments_rating'

# Establish a connection and execute the query
conn = psycopg2.connect(database=db, user=user, password=password, host=host, port=port)
cursor = conn.cursor()
cursor.execute(f"SELECT * FROM {table_name}")

# Fetch all rows from the cursor
rows = cursor.fetchall()

# Create a pandas DataFrame from the fetched rows
df_likes_dis = pd.DataFrame(rows, columns=[desc[0] for desc in cursor.description])

# Close the cursor and the connection
cursor.close()
conn.close()

# # Display the DataFrame
# df.head()

In [ ]:
df_likes_dis.head()

,id,test_id,user_id,video_url,rating,created_at
0,49,e6bec286-9971-47aa-acf3-c35de89afb9b,12,https://www.youtube.com/watch?v=NccsIt6dDPE,0,2023-06-30 14:46:15.178108+02:00
1,50,e6bec286-9971-47aa-acf3-c35de89afb9b,12,https://www.youtube.com/watch?v=FEm2ZzSFAv4,0,2023-06-30 14:46:20.008861+02:00
2,2,b0985c5a-89a9-4354-a1e6-0be2d2f3856f,3,https://www.youtube.com/watch?v=zVvE4_-ISUU,1,2023-06-30 10:49:00.511333+02:00
3,3,b0985c5a-89a9-4354-a1e6-0be2d2f3856f,3,https://www.youtube.com/watch?v=DM2XFuzhEuU,1,2023-06-30 10:51:52.558999+02:00
4,4,b0985c5a-89a9-4354-a1e6-0be2d2f3856f,3,https://www.youtube.com/watch?v=rLsimrBoYXc,1,2023-06-30 10:55:03.355728+02:00


In [ ]:
#Calculate the overall like/dislike ratio:
like_dislike_ratio = df_likes_dis['rating'].mean()
like_dislike_ratio

0.7444444444444445

In [ ]:
#Analyze the distribution of ratings:
rating_counts = df_likes_dis['rating'].value_counts()
rating_counts

1    134
0     46
Name: rating, dtype: int64

In [ ]:
#Identify the most liked or disliked videos:

most_liked_videos = df_likes_dis[df_likes_dis['rating'] == 1]['video_url'].value_counts().head(5)
most_disliked_videos = df_likes_dis[df_likes_dis['rating'] == 0]['video_url'].value_counts().head(5)


In [ ]:
most_liked_videos

https://www.youtube.com/watch?v=a7gFkUqIv1E    19
https://www.youtube.com/watch?v=11U0h0DPu7k    18
https://www.youtube.com/watch?v=SLEK4GdRXoM    14
https://www.youtube.com/watch?v=eiEMVA8AIJw    13
https://www.youtube.com/watch?v=JGW7ik4xse4     9
Name: video_url, dtype: int64

In [ ]:
most_disliked_videos

https://www.youtube.com/watch?v=NccsIt6dDPE    12
https://www.youtube.com/watch?v=a7gFkUqIv1E     5
https://www.youtube.com/watch?v=SLEK4GdRXoM     4
https://www.youtube.com/watch?v=eiEMVA8AIJw     4
https://www.youtube.com/watch?v=G88lq7VTTMw     4
Name: video_url, dtype: int64

In [ ]:
# average_rating_per_video = df_likes_dis.groupby('video_url')['rating'].mean()
# average_rating_per_video

In [ ]:
#high disagreement videos
video_rating_std = df_likes_dis.groupby('video_url')['rating'].std()
high_disagreement_videos = video_rating_std[video_rating_std > 0.5]
high_disagreement_videos

video_url
https://www.youtube.com/watch?v=G88lq7VTTMw    0.527046
https://www.youtube.com/watch?v=bP9bT6xfhNE    0.707107
https://www.youtube.com/watch?v=gLxDCC9qHRk    0.547723
Name: rating, dtype: float64

By identifying the videos with high disagreement in ratings, we can pinpoint videos that elicit mixed opinions from users. This indicates that different users have contrasting preferences or subjective judgments about these particular videos. It suggests that these videos may be more controversial or polarizing in terms of user satisfaction or engagement.